# Classifier training

In [103]:
import pandas as pd
import numpy as np
import tensorflow as tf
import scipy.stats as sm
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns

sns.set()

# Loading the data

In [104]:
df = pd.read_csv('data/out.csv')
print(df.head())

    age  sex  chest_pain  rest_blood_pressure  cholestorol  fast_blood_sugar  \
0  63.0  1.0         1.0                145.0        233.0               1.0   
1  67.0  1.0         4.0                160.0        286.0               0.0   
2  67.0  1.0         4.0                120.0        229.0               0.0   
3  37.0  1.0         3.0                130.0        250.0               0.0   
4  41.0  0.0         2.0                130.0        204.0               0.0   

   rest_ecg  max_heart_rate  exercise_angina  st_depression  ...    age_young  \
0       2.0           150.0              0.0            2.3  ...         True   
1       2.0           108.0              1.0            1.5  ...         True   
2       2.0           129.0              1.0            2.6  ...         True   
3       0.0           187.0              0.0            3.5  ...        False   
4       2.0           172.0              0.0            1.4  ...        False   

   age_middle   bp_1   bp_2   bp

In [105]:
y = df["target"]
X = df.drop("target", axis=1)

## Split data
Splitting the data into two sets (`train` and `test`) means that the test set can be used as a new, unbiased dataset to evaluate the performance of the model against. This also helps detect the phenomenon of overfitting, where a model is extremely tailored to the training dataset, and therefore performs poorly on unseen datasets.

In this case, 80% of the samples will go to the training set, and the remaining 20 will go to the test set.

In [106]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)
print("X_train:", X_train.shape, "X_test:", X_test.shape)
print("y_train:", y_train.shape, "y_test:", y_test.shape)

X_train: (736, 24) X_test: (184, 24)
y_train: (736,) y_test: (184,)


### Dummy Variables


In [107]:
df_train = pd.get_dummies(data=X_train, drop_first=True)
df_test = pd.get_dummies(data=X_test, drop_first=True)

## Scale data
Variables with much broader ranges of values might disproportionately effect the final model, so standardising the data ensures that all features are proportionately weighted. 

The formula used for feature standardisation transforms each feature to have zero-mean and unit-variance:

$ \large{x' = \frac{x - \overline{x}}{\sigma}} $

Where $x$ is the original feature vector, $ \bar{x}=average(x)$ is the mean of that feature vector, and $\sigma$ is its standard deviation.

In [108]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype bool, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda3/lib/python3.6/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype bool, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: DataConversionWarning: Data with input dtype bool, float64 were all converted to float64 by StandardScaler.
  """


# Training the classifiers
The following models will be compared:

Using `sklearn`: 
- Decision tree 
- Support Vector Machine
- Logistic Regression
- Random Forest

Using `Tensorflow`:
- Artificial Neural Network

## Decision Tree classifier

In [109]:
def get_accuracy():
    print("Sklearn accuracy:", sklearn.metrics.accuracy_score(y_test, y_pred))
    print("Sklearn recall:", sklearn.metrics.recall_score(y_test, y_pred, average='micro'))
    print("\nConfusion matrix (predicted vs actual)")
    print(pd.DataFrame(np.c_[sklearn.metrics.confusion_matrix(y_test, y_pred)]))  

In [110]:
from sklearn.tree import DecisionTreeClassifier  

# Create a decision tree Classifier
classifier = DecisionTreeClassifier()  

# Fit the model
classifier.fit(X_train, y_train)  

# Predict the response for test dataset
y_pred = classifier.predict(X_test)  

# Get accuracy results
get_accuracy()

Sklearn accuracy: 0.5163043478260869
Sklearn recall: 0.5163043478260869

Confusion matrix (predicted vs actual)
    0   1  2  3  4
0  69   6  5  6  2
1   9  11  6  8  2
2   8   6  7  4  1
3   5   5  5  7  3
4   1   1  4  2  1


### Hyperparamater tuning
Testing combinations of possible hyperparameters to attempt to improve the model's recall search. This is done with `Random Search` over `Grid Search` for efficiency

In [111]:
from sklearn.model_selection import RandomizedSearchCV

param_grid = {
    'max_depth': range(1, 10),
    'splitter': ['best', 'random'],
    'criterion': ['gini', 'entropy'],
    'max_depth': range(2, 15),
    'min_samples_split': range(2, 5),
    'min_samples_leaf': range(2, 5)
}

rs = RandomizedSearchCV(
    estimator=DecisionTreeClassifier(),
    iid=False,
    return_train_score=True,
    param_distributions=param_grid, 
    scoring='recall_micro', 
    cv=10, 
    n_jobs=-1)

%time rs = rs.fit(X_train, y_train)
print("Score:", rs.best_score_)
print("Best parameters:", rs.best_params_)

CPU times: user 201 ms, sys: 80.2 ms, total: 281 ms
Wall time: 3.41 s
Score: 0.4834708338475462
Best parameters: {'splitter': 'best', 'min_samples_split': 2, 'min_samples_leaf': 3, 'max_depth': 5, 'criterion': 'gini'}


## Support Vector Machine

In [112]:
from sklearn.svm import SVC

# Create an SVM Classifier
classifier = SVC(kernel='linear') # Linear Kernel

# Fit the model
classifier.fit(X_train, y_train)  

# Train the model and predict against the validation set
y_pred = classifier.predict(X_test)  

# Get accuracy results
get_accuracy()

Sklearn accuracy: 0.532608695652174
Sklearn recall: 0.532608695652174

Confusion matrix (predicted vs actual)
    0   1  2  3  4
0  73   5  3  7  0
1  14  10  7  5  0
2   4   9  6  7  0
3   3   7  6  9  0
4   1   1  4  3  0


### Hyperparameter tuning

In [113]:
param_range = np.logspace(-1, 1, 10)
param_grid = {
        'C': param_range,
        'gamma': param_range,
        'kernel': ['linear'],
        'degree': range(1, 7)
}

rs = RandomizedSearchCV(
    estimator=SVC(),
    iid=False,
    return_train_score=True,
    param_distributions=param_grid, 
    scoring='recall_micro', 
    cv=10, 
    n_jobs=-1)

%time rs = rs.fit(X_train, y_train)
print("Score:", rs.best_score_)
print("Best parameters:", rs.best_params_)

CPU times: user 315 ms, sys: 38.2 ms, total: 353 ms
Wall time: 5.05 s
Score: 0.5174056727960836
Best parameters: {'kernel': 'linear', 'gamma': 3.593813663804626, 'degree': 1, 'C': 0.2782559402207124}


## Logistic regression

In [114]:
from sklearn.linear_model import LogisticRegression

classifier = LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial')

# Fit the model
classifier.fit(X_train, y_train)  

# Predict the response for test dataset
y_pred = classifier.predict(X_test)  

# Get accuracy results
get_accuracy()

Sklearn accuracy: 0.5271739130434783
Sklearn recall: 0.5271739130434783

Confusion matrix (predicted vs actual)
    0   1  2  3  4
0  72   8  1  7  0
1  14  12  6  4  0
2   4  10  4  8  0
3   3   8  6  8  0
4   1   2  2  3  1


### Hyperparameter tuning

In [115]:
param_range = np.logspace(-2, 2, 10)
param_grid = {
    'C': param_range,
    'penalty': ['l1', 'l2']
}

rs = RandomizedSearchCV(
    estimator=LogisticRegression(solver='liblinear', multi_class='auto'),
    iid=False,
    return_train_score=True,
    param_distributions=param_grid, 
    scoring='recall_micro', 
    cv=10, 
    n_jobs=-1)

%time rs = rs.fit(X_train, y_train)
print("Score:", rs.best_score_)
print("Best parameters:", rs.best_params_)

CPU times: user 215 ms, sys: 27.5 ms, total: 243 ms
Wall time: 2.24 s
Score: 0.528145398823481
Best parameters: {'penalty': 'l1', 'C': 0.21544346900318834}


## Random Forest

In [116]:
from sklearn.ensemble import RandomForestClassifier

classifier = clf = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0)

# Fit the model
classifier.fit(X_train, y_train)  

# Predict the response for test dataset
y_pred = classifier.predict(X_test)  

# Get accuracy results
get_accuracy()

Sklearn accuracy: 0.5108695652173914
Sklearn recall: 0.5108695652173914

Confusion matrix (predicted vs actual)
    0  1  2  3  4
0  87  1  0  0  0
1  29  7  0  0  0
2  19  7  0  0  0
3  16  9  0  0  0
4   3  6  0  0  0


### Hyperparameter tuning

In [117]:
param_range = np.logspace(-6, 4, 20)
param_grid = {
    'max_depth': [10, 50, 100],
    'n_estimators': range(1, 10),
    'criterion': ['gini', 'entropy'],
    'max_depth': range(2, 15),
    'min_samples_split': range(2, 5),
    'min_samples_leaf': range(2, 5)
}

rs = RandomizedSearchCV(
    estimator=RandomForestClassifier(),
    iid=False,
    return_train_score=True,
    param_distributions=param_grid, 
    scoring='recall_micro', 
    cv=10, 
    n_jobs=-1)

%time rs = rs.fit(X_train, y_train)
print("Score:", rs.best_score_)
print("Best parameters:", rs.best_params_)

CPU times: user 177 ms, sys: 14 ms, total: 191 ms
Wall time: 853 ms
Score: 0.5161648977744869
Best parameters: {'n_estimators': 9, 'min_samples_split': 4, 'min_samples_leaf': 4, 'max_depth': 5, 'criterion': 'entropy'}


**Comments**:
- Logistic regresion performs the best, achieving the highest recall score (around `0.58`)
- Hyperparameter tuning improves the score slightly. Better results could be achieved by adding wider parameter searches or performing Grid Search but these would need exponentially more compute power

## Neural Network

The Neural Network was built with trial and error, until the best performing network was found. 
It contains:
- 13 input nodes, which correspond to the 13 input features
- 6 nodes in the first hidden layer, which uses the `ReLU` activation function and the `uniform` bias initialiser
- 13 nodes in the second hidden layer, which also uses the `ReLU` activation function
- 1 output layer with 4 nodes, which uses the `Softmax` activation function. This layer outputs a prediction of the level of severity of heart disease (`0` = absence, `1-4` = presence) 
- Categorical cross entropy (`categorical_crossentropy`) for calculating the loss function
- Adam (`adam`) algorithm for optimising the loss function

<img src="data/nn3.png" width="600" />

In [118]:
df = pd.read_csv('data/out.csv')

drop = ['target', 'age_mature', 'age_young', 'age_middle', 'bp_1', 'bp_2', 'bp_3', 'bp_4',
       'ch_1', 'ch_2', 'ch_3', 'ch_4']

y = df["target"]
X = df.drop(drop, axis=1)

In [119]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(sparse=False,  categories='auto')
y = encoder.fit_transform(y.values.reshape(-1, 1))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)
print("X_train:", X_train.shape, "X_test:", X_test.shape)
print("y_train:", y_train.shape, "y_test:", y_test.shape)

X_train: (736, 13) X_test: (184, 13)
y_train: (736, 5) y_test: (184, 5)


In [120]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [121]:
import tensorflow as tf
from tensorflow import keras

classifier = keras.Sequential()

# adding the input layer (input_dim) and the first hidden layer
classifier.add(keras.layers.Dense(6, bias_initializer = 'uniform', activation = 'relu', input_dim = X_train.shape[1], name='hidden_1'))

# adding the second and third hidden layers
classifier.add(keras.layers.Dense(13, activation='relu', name='hidden_2'))

# adding the output layer
classifier.add(keras.layers.Dense(y_train.shape[1], activation='softmax', name='output'))


In [122]:
# compiling the neural network
# classifier.compile(optimizer = 'sgd', loss = 'mean_squared_error', metrics = ['recall'])
classifier.compile(loss = 'categorical_crossentropy' , optimizer = 'adam' , metrics = ['accuracy'] )


In [123]:
classifier.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
hidden_1 (Dense)             (None, 6)                 84        
_________________________________________________________________
hidden_2 (Dense)             (None, 13)                91        
_________________________________________________________________
output (Dense)               (None, 5)                 70        
Total params: 245
Trainable params: 245
Non-trainable params: 0
_________________________________________________________________


In [124]:
# fitting the model 
classifier.fit(X_train, y_train, batch_size = 10, epochs = 100)

Epoch 1/100
736/736 [==============================] - 1s 832us/step - loss: 1.6815 - acc: 0.2582
Epoch 2/100
736/736 [==============================] - 0s 226us/step - loss: 1.5014 - acc: 0.3845
Epoch 3/100
736/736 [==============================] - 0s 223us/step - loss: 1.3792 - acc: 0.4986
Epoch 4/100
736/736 [==============================] - 0s 150us/step - loss: 1.2948 - acc: 0.4918
Epoch 5/100
736/736 [==============================] - 0s 151us/step - loss: 1.2290 - acc: 0.5054
Epoch 6/100
736/736 [==============================] - 0s 207us/step - loss: 1.1854 - acc: 0.5149
Epoch 7/100
736/736 [==============================] - 0s 186us/step - loss: 1.1566 - acc: 0.5217
Epoch 8/100
736/736 [==============================] - 0s 180us/step - loss: 1.1367 - acc: 0.5476
Epoch 9/100
736/736 [==============================] - 0s 180us/step - loss: 1.1207 - acc: 0.5435
Epoch 10/100
736/736 [==============================] - 0s 180us/step - loss: 1.1085 - acc: 0.5367
Epoch 11/100
736/73

736/736 [==============================] - 0s 173us/step - loss: 0.9818 - acc: 0.5951
Epoch 84/100
736/736 [==============================] - 0s 159us/step - loss: 0.9809 - acc: 0.5951
Epoch 85/100
736/736 [==============================] - 0s 155us/step - loss: 0.9808 - acc: 0.5992
Epoch 86/100
736/736 [==============================] - 0s 170us/step - loss: 0.9802 - acc: 0.5938
Epoch 87/100
736/736 [==============================] - 0s 167us/step - loss: 0.9796 - acc: 0.5910
Epoch 88/100
736/736 [==============================] - 0s 187us/step - loss: 0.9785 - acc: 0.5897
Epoch 89/100
736/736 [==============================] - 0s 198us/step - loss: 0.9770 - acc: 0.5938
Epoch 90/100
736/736 [==============================] - 0s 210us/step - loss: 0.9783 - acc: 0.6019
Epoch 91/100
736/736 [==============================] - 0s 161us/step - loss: 0.9761 - acc: 0.5951
Epoch 92/100
736/736 [==============================] - 0s 198us/step - loss: 0.9767 - acc: 0.5978
Epoch 93/100
736/736 [=

In [125]:
results = classifier.evaluate(X_test, y_test)

print('Final test set loss: {:4f}'.format(results[0]))
print('Final test set accuracy: {:4f}'.format(results[1]))

184/184 [==============================] - 0s 702us/step
Final test set loss: 1.284249
Final test set accuracy: 0.472826


In [126]:
from sklearn.metrics import confusion_matrix

y_pred = classifier.predict(X_test)
cm = confusion_matrix(y_test.argmax(axis=1), y_pred.argmax(axis=1))
print(cm)

[[64 11  2  2  0]
 [14 11  6  9  0]
 [ 5  7  3 14  0]
 [ 7  3  7  9  0]
 [ 1  4  1  4  0]]


In [127]:
def print_cm(cm, labels, hide_zeroes=False, hide_diagonal=False, hide_threshold=None):
    """pretty print for confusion matrixes"""
    columnwidth = max([len(x) for x in labels] + [5])  # 5 is value length
    empty_cell = " " * columnwidth
    # Print header
    print("    " + empty_cell, end=" ")
    for label in labels:
        print("%{0}s".format(columnwidth) % label, end=" ")
    print()
    # Print rows
    for i, label1 in enumerate(labels):
        print("    %{0}s".format(columnwidth) % label1, end=" ")
        for j in range(len(labels)):
            cell = "%{0}.1f".format(columnwidth) % cm[i, j]
            if hide_zeroes:
                cell = cell if float(cm[i, j]) != 0 else empty_cell
            if hide_diagonal:
                cell = cell if i != j else empty_cell
            if hide_threshold:
                cell = cell if cm[i, j] > hide_threshold else empty_cell
            print(cell, end=" ")
        print()

In [128]:
print_cm(cm, ["0", "1", "2", "3", "4"])

              0     1     2     3     4 
        0  64.0  11.0   2.0   2.0   0.0 
        1  14.0  11.0   6.0   9.0   0.0 
        2   5.0   7.0   3.0  14.0   0.0 
        3   7.0   3.0   7.0   9.0   0.0 
        4   1.0   4.0   1.0   4.0   0.0 


In [129]:
import tensorflowjs as tfjs
tfjs.converters.save_keras_model(classifier, ".")